In [1]:
import os
os.chdir(r"C:\Users\PC\Desktop\work\projectML\kidney-disease-classification-MLflow")
%pwd

'C:\\Users\\PC\\Desktop\\work\\projectML\\kidney-disease-classification-MLflow'

In [2]:
import dagshub
dagshub.init(repo_owner='mouaadhalimi', repo_name='kidney-disease-classification-MLflow', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as mouaadhalimi

Initialized MLflow to track repo "mouaadhalimi/kidney-disease-classification-MLflow"

Repository mouaadhalimi/kidney-disease-classification-MLflow initialized!

🏃 View run amusing-stag-578 at: https://dagshub.com/mouaadhalimi/kidney-disease-classification-MLflow.mlflow/#/experiments/0/runs/24c2a4c22dea4fa5813a4549d96fa820
🧪 View experiment at: https://dagshub.com/mouaadhalimi/kidney-disease-classification-MLflow.mlflow/#/experiments/0


In [3]:
from dataclasses import dataclass
from pathlib import Path
import tensorflow as tf

In [4]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [5]:
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [6]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import create_directories, read_yaml, save_json

In [7]:




class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_evalution_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/mouaadhalimi/kidney-disease-classification-MLflow.mlflow",
            all_params=self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [8]:
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config = config
    def _valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )
        dataflow_kwargs = dict(
            target_size =self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
        **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
        directory = self.config.training_data,
        subset="validation",
        shuffle=False,
        **dataflow_kwargs
        )
        
    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples //  self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs= self.config.params_epochs,
            steps_per_epoch= self.steps_per_epoch,
            validation_steps= self.validation_steps,
            validation_data=self.valid_generator,
          
        )
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)
    
    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy":self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_artifact_uri()).scheme

        if mlflow.active_run():
            mlflow.end_run()

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy":self.score[1]}

            )
            if tracking_uri_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [10]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evalution_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

Found 93 images belonging to 2 classes.


c:\Users\PC\anaconda3\envs\project2\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 427ms/step - accuracy: 0.7221 - loss: 0.8014
🏃 View run sneaky-moose-666 at: https://dagshub.com/mouaadhalimi/kidney-disease-classification-MLflow.mlflow/#/experiments/0/runs/d9ff5c79c29c4bd9a72254e1bf01c79d
🧪 View experiment at: https://dagshub.com/mouaadhalimi/kidney-disease-classification-MLflow.mlflow/#/experiments/0


2025/03/28 19:12:53 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/03/28 19:13:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'VGG16Model'.
2025/03/28 19:15:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


🏃 View run handsome-penguin-920 at: https://dagshub.com/mouaadhalimi/kidney-disease-classification-MLflow.mlflow/#/experiments/0/runs/4245bb09e8ab40ff85f9b589b4021e4f
🧪 View experiment at: https://dagshub.com/mouaadhalimi/kidney-disease-classification-MLflow.mlflow/#/experiments/0
